In [243]:
#@title Hints
#Inspiration

#1.Convert the height and weight columns to numerical forms

#2.Remove the unnecessary newline characters from all columns that have them.

#3.Based on the 'Joined' column, check which players have been playing at a club for more than 10 years!

#4.'Value', 'Wage' and "Release Clause' are string columns. Convert them to numbers. For eg, "M" in value column is Million, so multiply the row values by 1,000,000, etc.

#5.Some columns have 'star' characters. Strip those columns of these stars and make the columns numerical

#6.Which players are highly valuable but still underpaid (on low wages)? (hint: scatter plot between wage and value)
#Ask more questions yourself !

#Hope it helps! :) If you like this dataset, please show your support by upvoting this dataset! Thanks! :)


In [244]:
!pip install pandasql

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [245]:
# import all libary we gonna use
import pandas as pd
import numpy as np
import pandasql as ps
from google.colab import data_table
from datetime import datetime
import re
import matplotlib.pyplot as plt
data_table.enable_dataframe_formatter()

In [246]:
# read csv file and redirect to "raw"
raw = pd.read_csv('/content/drive/MyDrive/Road to Data N/CleanData_practice/data/fifa21 raw data v2.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (76) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [247]:
#@title Query Zone
query = """ select Joined
            from raw
            limit 10
        """
result = ps.sqldf(query)
result

,Joined
0,"Jul 1, 2004"
1,"Jul 10, 2018"
2,"Jul 16, 2014"
3,"Aug 30, 2015"
4,"Aug 3, 2017"
5,"Jul 1, 2014"
6,"Jul 1, 2017"
7,"Jul 19, 2018"
8,"Jul 1, 2018"
9,"Jul 1, 2014"


In [248]:
#@title Columns
raw.columns

Index(['ID', 'Name', 'LongName', 'photoUrl', 'playerUrl', 'Nationality', 'Age',
       '↓OVA', 'POT', 'Club', 'Contract', 'Positions', 'Height', 'Weight',
       'Preferred Foot', 'BOV', 'Best Position', 'Joined', 'Loan Date End',
       'Value', 'Wage', 'Release Clause', 'Attacking', 'Crossing', 'Finishing',
       'Heading Accuracy', 'Short Passing', 'Volleys', 'Skill', 'Dribbling',
       'Curve', 'FK Accuracy', 'Long Passing', 'Ball Control', 'Movement',
       'Acceleration', 'Sprint Speed', 'Agility', 'Reactions', 'Balance',
       'Power', 'Shot Power', 'Jumping', 'Stamina', 'Strength', 'Long Shots',
       'Mentality', 'Aggression', 'Interceptions', 'Positioning', 'Vision',
       'Penalties', 'Composure', 'Defending', 'Marking', 'Standing Tackle',
       'Sliding Tackle', 'Goalkeeping', 'GK Diving', 'GK Handling',
       'GK Kicking', 'GK Positioning', 'GK Reflexes', 'Total Stats',
       'Base Stats', 'W/F', 'SM', 'A/W', 'D/W', 'IR', 'PAC', 'SHO', 'PAS',
       'DRI', 'DEF', 

In [249]:
#@title missing value
#check the columns that have missing value
print('rows: '+str(raw.shape[0])+'\ncolumns: '+str(raw.shape[1]))
raw.isna().sum()[raw.isna().sum() != 0]

rows: 18979
columns: 77


Loan Date End    17966
Hits              2595
dtype: int64

In [250]:
# convert date column to date format
raw["Joined"] = pd.to_datetime(raw["Joined"], format="%b %d, %Y").dt.date
raw["Loan Date End"] = pd.to_datetime(raw["Loan Date End"], format="%b %d, %Y").dt.date

In [251]:
raw['month'] = raw['Joined'].month

AttributeError: ignored

# Duplicated value

In [ ]:
#check duplicated player using LongName, Height, Nationality, and Age
raw[raw[['LongName', 'Height', 'Nationality', 'Age']].duplicated(keep=False)].loc[:,['Name', 'LongName', 'Height', 'Age', 'Nationality', 'Club', 'POT', 'Best Position', 'photoUrl']].sort_values(['LongName', 'Age'])

from previous result, i found that the player name "Peng Wang" has 2 duplicated record. So, i choose to drop his lower performance record

In [ ]:
#drop lower performance row
raw = raw.sort_values(['LongName', 'POT']).drop_duplicates(subset=['LongName', 'Height', 'Nationality', 'Age'], keep='last')

In [ ]:
#explore some numeric column
raw[['Height', 'Weight', 'Value', 'Wage']].info()

**All column above shoud be converted to numeric column (see insight data for each column using Query zone)**

# Height

---



In [ ]:
# create function for converting Height to cm in number
def to_cm(high):
  if 'cm' in high:
    return float(re.sub('cm','',high))
  elif "'" in high:
    high = high.split("'")
    feet = float(high[0])
    inch = float(re.sub('"','',high[1]))
    return feet*30.48 + inch*2.54

In [ ]:
#convert Height column 
raw['Height'] = raw['Height'].apply(lambda x : to_cm(x))

In [ ]:
#check Height column
raw[['Height']].describe()

# Weight

---



In [ ]:
# create function to convert weight to kg in number
def to_kg(mass):
  if 'kg' in mass:
    return float(re.sub('kg', '', mass))
  elif 'lbs' in mass:
    return round(float(re.sub('lbs', '', mass))/2.205, 2)

In [ ]:
# convert Weight column
raw['Weight'] = raw['Weight'].apply(lambda x: to_kg(x))

In [ ]:
raw[['Weight']].describe()

# Value&Wage

---


In [ ]:
# create function to convert to be amount of money in number
def to_num(pennie):
  pennie = re.sub('€', '', pennie)
  if 'M' in pennie:
    pennie = re.sub('M', '', pennie)
    return float(pennie)*1000000
  elif 'K' in pennie:
    pennie = re.sub('K', '', pennie)
    return float(pennie)*1000
  else:
    return float(pennie)

In [ ]:
# convert money columns
raw['Value'] = raw['Value'].apply(lambda x: to_num(x))
raw['Wage'] = raw['Wage'].apply(lambda x: to_num(x))
raw['Release Clause'] = raw['Release Clause'].apply(lambda x: to_num(x))

In [ ]:
# check money column
raw[['Value', 'Wage', 'Release Clause']].describe()

In [ ]:
# create color string array 
color=[]
for value in raw['Wage']:
  if value < 1e5:
    color.append('black')
  elif value < 2*1e5:
    color.append('blue')
  else:
    color.append('red')

In [ ]:
# dive into more detail about these couple columns
import matplotlib
plt.scatter(raw['Value']/1e8, 
            raw['Wage'],
            s=2.5,
            c=color)

#plt.gca.set_xscale('log')
plt.grid(True, which="both")
plt.title('Value and Wage relation')
plt.xlabel('Value in euro (Million)')
plt.ylabel('Wage per week in euro')
plt.figure(figsize=(12,8))
plt.show()